In [31]:
#parentPath = os.path.abspath('.')
#from RedditDJIA import CleanRedditNewsDJIA

#clean = CleanRedditNewsDJIA('../Data/Reddit_News_DJIA.csv')
#clean.cleanUp()

In [32]:
import os, sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchtext 


In [33]:
parentPath = os.path.abspath('.')
sys.path.append(parentPath)
from RedditDJIA import RedditNewsDJIADataset, ToTensor
train_newsData = RedditNewsDJIADataset('../Data/Reddit_News_DJIA_clean.csv', True)
for i, row in enumerate(train_newsData):
    print(i, "Label =", row['label'], "Features =", row['features'][:2])

    if i == 3:
        break

test_newsData = RedditNewsDJIADataset('../Data/Reddit_News_DJIA_clean.csv', False)




0 Label = 0 Features = ["Georgia 'downs two Russian warplanes' as countries move to brink of war"
 'BREAKING: Musharraf to be impeached.']
1 Label = 1 Features = ['Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'
 'Bush puts foot down on Georgian conflict']
2 Label = 0 Features = ['Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too.'
 "Russia 'ends Georgia operation"]
3 Label = 0 Features = ['U.S. refuses Israel weapons to attack Iran: report'
 "When the president ordered to attack Tskhinvali [the capital of South Ossetia], we knew then we were doomed. How come he didn't realize that?"]


In [34]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
#train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for set in data_iter:
        #print(set)

        for feature in set['features']:
            #print(feature)
            yield tokenizer(feature)

vocab = build_vocab_from_iterator(yield_tokens(train_newsData), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [35]:
vocab(['here', 'is', 'an', 'example'])


[490, 13, 26, 2597]

In [36]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)
label_pipeline('1')

1

In [37]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    #print('batch')
    label_list, text_list, offsets = [], [], [0]
    for set in batch:
        #print(set)
        label_list.append(label_pipeline(set['label']))
        row_text = []
        for _text in set['features']:
            processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
            row_text.append(processed_text)
        row_text = torch.cat(row_text)
        text_list.append(row_text)
        offsets.append(row_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1], dtype=torch.int64).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_newsData = RedditNewsDJIADataset('../Data/Reddit_News_DJIA_clean.csv', True)
dataloader = DataLoader(train_newsData, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [38]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        #print(text.shape,offsets.shape)
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [39]:

num_class = len(set([set['label'] for set in train_newsData]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

print('Number of Classes =', num_class)
print('Number of Vocab =', vocab_size)

Number of Classes = 2
Number of Vocab = 36062


In [40]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        #print(text, offsets)
        predicted_label = model(text, offsets)
        #print(predicted_label.shape, label.shape)
        #print(predicted_label[0], label[0])
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [41]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
#train_iter, test_iter = AG_NEWS()
train_iter = RedditNewsDJIADataset('../Data/Reddit_News_DJIA_clean.csv', True)
test_iter = RedditNewsDJIADataset('../Data/Reddit_News_DJIA_clean.csv', False)



train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)

split_train_, split_valid_ =  random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)



valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.78s | valid accuracy    0.500 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.79s | valid accuracy    0.500 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.78s | valid accuracy    0.537 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.80s | valid accuracy    0.500 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.78s | valid accuracy    0.500 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.79s |

In [42]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.508
